In [24]:
# import labraries
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import os
import re

In [25]:
# set up chrome driver
options = Options()
options.add_argument('--start-maximized')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')

In [26]:
service = Service("..\\chromedriver-win64\\chromedriver.exe")
browser = webdriver.Chrome(service=service, options=options)

In [27]:
# Go to the job detail page

browser.get('https://www.irishjobs.ie/jobs/data-analyst?searchOrigin=Homepage_top-search')
time.sleep(3) 

In [32]:
# Store all job info here
job_details = []

# Extract experience requirement from page content
def extract_experience(content):
    # Mapping of English number words to Arabic numerals
    word_to_num = {
        'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5',
        'six': '6', 'seven': '7', 'eight': '8', 'nine': '9', 'ten': '10',
        'eleven': '11', 'twelve': '12', 'fifteen': '15', 'twenty': '20'
    }
    
    # Expanded patterns to match various experience formats
    patterns = [
        # Standard patterns with 'experience'
        r'(\d+)\s*[-+]?\s*years?\s*experience',
        r'(\d+)\s*years?\s*of\s*experience',
        r'(\d+)\s*experience',
        r'(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|fifteen|twenty)\s*years?\s*experience',
        r'(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|fifteen|twenty)\s*years?\s*of\s*experience',
        r'(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|fifteen|twenty)\s*experience',
        
        # Patterns without 'experience' keyword
        r'(\d+)\s*[-+]?\s*years?\s*in',
        r'(\d+)\s*[-+]?\s*years?\s*working',
        r'(\d+)\s*[-+]?\s*years?\s*background',
        r'(\d+)\s*[-+]?\s*years?\s*relevant',
        r'(\d+)\s*[-+]?\s*years?\s*professional',
        r'(\d+)\s*[-+]?\s*years?\s*industry',
        
        # Minimum/maximum patterns
        r'minimum\s*(\d+)\s*years?',
        r'at\s*least\s*(\d+)\s*years?',
        r'minimum\s*of\s*(\d+)\s*years?',
        r'(\d+)\s*[-+]\s*years?\s*minimum',
        r'(\d+)\s*to\s*(\d+)\s*years?',
        
        # Range patterns (take the lower number)
        r'(\d+)\s*[-–]\s*(\d+)\s*years?',
        
        # English number words with various contexts
        r'(one|two|three|four|five|six|seven|eight|nine|ten)\s*years?\s*in',
        r'(one|two|three|four|five|six|seven|eight|nine|ten)\s*years?\s*working',
        r'(one|two|three|four|five|six|seven|eight|nine|ten)\s*years?\s*background',
        r'(one|two|three|four|five|six|seven|eight|nine|ten)\s*years?\s*relevant',
        
        # Additional patterns
        r'(\d+)\+\s*years?',
        r'over\s*(\d+)\s*years?',
        r'more\s*than\s*(\d+)\s*years?',
        r'(\d+)\s*or\s*more\s*years?'
    ]
    
    # Search for patterns in content
    for pattern in patterns:
        matches = re.findall(pattern, content, re.IGNORECASE)
        if matches:
            # Handle different match types
            if isinstance(matches[0], tuple):
                # For range patterns, take the first (lower) number
                value = matches[0][0].lower()
            else:
                value = matches[0].lower()
            
            # Convert English number words to Arabic numerals
            if value in word_to_num:
                return word_to_num[value]
            # Return Arabic numeral if already numeric
            elif value.isdigit():
                return value
    
    # Fallback logic for general experience levels
    if re.search(r'entry.level|junior|graduate|fresh|trainee|intern', content, re.IGNORECASE):
        return "0"  # Entry Level = 0
    elif re.search(r'senior|lead|principal|manager|director', content, re.IGNORECASE):
        return "5"  # Senior Level = 5
    elif re.search(r'mid.level|intermediate|experienced', content, re.IGNORECASE):
        return "3"  # Mid Level = 3
    
    return "0"  # Not specified = 0
    
    # Fallback logic for general experience levels
    if re.search(r'entry.level|junior|graduate|fresh', content, re.IGNORECASE):
        return "0"  # Entry Level = 0
    elif re.search(r'senior|lead|principal', content, re.IGNORECASE):
        return "5"  # Senior Level = 5
    elif re.search(r'mid.level|intermediate', content, re.IGNORECASE):
        return "3"  # Mid Level = 3
    return "0"  # Not specified = 0

# Extract skills requirement from page content
def extract_skills(content):
    skill_keywords = [
        'Python', 'R', 'SQL', 'Excel', 'Power BI', 'Tableau', 
        'Machine Learning', 'Statistics', 'Data Visualization',
        'Pandas', 'NumPy', 'Matplotlib', 'Seaborn', 'Jupyter',
        'AWS', 'Azure', 'Google Cloud', 'Spark', 'Hadoop',
        'SAS', 'SPSS', 'Looker', 'Qlik', 'PowerPoint'
    ]
    found_skills = []
    for skill in skill_keywords:
        if re.search(rf'\b{skill}\b', content, re.IGNORECASE):
            found_skills.append(skill)
    return ', '.join(found_skills) if found_skills else "Not specified"

# Run this function for each page after manual page turn
# Collect job info from current page
def scrape_current_page():
    # Re-find job cards to avoid stale element reference
    job_cards = browser.find_elements(By.CSS_SELECTOR, 'article.res-4cwuay')
    print(f'Found {len(job_cards)} jobs on this page')
    
    for i in range(len(job_cards)):
        try:
            # Re-find job cards each time to avoid stale element
            job_cards = browser.find_elements(By.CSS_SELECTOR, 'article.res-4cwuay')
            if i >= len(job_cards):
                break
                
            card = job_cards[i]
            print(f'Processing job {i+1} ...')
            
            # Click the job card
            card.click()
            time.sleep(3)
            
            # Get page content
            page_content = browser.page_source
            
            # Extract information
            experience = extract_experience(page_content)
            skills = extract_skills(page_content)
            
            # Get job title and company
            try:
                job_title = browser.find_element(By.CSS_SELECTOR, 'h1').text
            except:
                job_title = 'N/A'
            
            try:
                company = browser.find_element(By.CSS_SELECTOR, '[data-testid="company-name"]').text
            except:
                company = 'N/A'
            
            # Store the data
            job_details.append({
                'job_title': job_title,
                'company': company,
                'experience': experience,
                'skills': skills
            })
            
            print(f'  - Title: {job_title}')
            print(f'  - Company: {company}')
            print(f'  - Experience: {experience}')
            print(f'  - Skills: {skills[:50]}...' if len(skills) > 50 else f'  - Skills: {skills}')
            
            # Go back to job list
            browser.back()
            time.sleep(3)
            
        except Exception as e:
            print(f'Error on job {i+1}: {e}')
            try:
                browser.back()
                time.sleep(2)
            except:
                pass
            continue

# Save data to CSV (追加模式)
def save_data():
    if job_details:
        df = pd.DataFrame(job_details)
        filename = '../datasets/raw/DA_experience_skills.csv'
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        # Check if file exists to decide whether to write header
        file_exists = os.path.isfile(filename)
        
        # Append to CSV (mode='a' for append)
        df.to_csv(filename, mode='a', header=not file_exists, index=False, encoding='utf-8')
        
        print(f'Data appended to {filename}')
        print(f'Current batch: {len(job_details)} jobs')
        
        # Clear the job_details list for next page
        job_details.clear()
        print('job_details cleared for next page')
    else:
        print('No data to save')

# Execute the scraping for current page
print("Starting to scrape current page...")
scrape_current_page()
print("\nCurrent page scraping completed!")
print(f"Total jobs collected so far: {len(job_details)}")
save_data()

Starting to scrape current page...
Found 5 jobs on this page
Processing job 1 ...


  - Title: CREDit Graduate Program Analyst - Structured Finance- Dublin
  - Company: N/A
  - Experience: 0
  - Skills: Python, R, Excel, Machine Learning, Statistics, Po...
Processing job 2 ...
  - Title: Associate Sales Support & Administration Analyst - IRE F
  - Company: N/A
  - Experience: 0
  - Skills: R, SQL
Processing job 3 ...
  - Title: Weekend Shift- Forensic and Incident Response Operations (FIRE) Analyst
  - Company: N/A
  - Experience: 0
  - Skills: Python, R
Processing job 4 ...
  - Title: Snr Data Team Lead
  - Company: N/A
  - Experience: 5
  - Skills: R
Processing job 5 ...
  - Title: Senior Clinical Data Team Lead (Senior DTL) - FSP
  - Company: N/A
  - Experience: 7
  - Skills: R, Excel, PowerPoint

Current page scraping completed!
Total jobs collected so far: 5
Data appended to ../datasets/raw/DA_experience_skills.csv
Current batch: 5 jobs
job_details cleared for next page
